In [3]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
import warnings
 
warnings.filterwarnings("ignore")

In [4]:
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

In [3]:
#train_dataset = train_dataset.batch(512).prefetch(1)
#train_dataset.shape()

In [12]:

train_path = 'train'
train_files = sorted(glob.glob(train_path + '/*'))
X_data=[]
land_type=(0,1,2,3)
for file in train_files:
    dataset = np.load(file)
        
    target= dataset[:,:,-1].reshape(40,40,1)
    cutoff_labels = np.where(target < 0, 0, target)
    feature = dataset[:,:,9:10]
    if (cutoff_labels > 0).sum() < 50: 
        continue
    cut_data=feature+cutoff_labels
    X_data.append(cut_data)


KeyboardInterrupt: 

In [90]:
print(X_data.shape)
np.random.shuffle(X_data)


TypeError: 'tuple' object is not callable

In [91]:
print(X_train.shape)
print(Y_train.shape)

(30760, 40, 40, 1)
(30760, 40, 40, 1)


In [93]:

X_test1=X_train[0:1000,:,:,:].copy()
X_train1=X_train[1000:30761,:,:,:].copy()
Y_test1=Y_train[0:1000,:,:,:].copy()
Y_train1=Y_train[1000:30761,:,:,:].copy()

X_test1=X_test1.reshape(1000*40*40,1)
Y_test1=Y_test1.reshape(1000*40*40,1)
X_train1=X_train1.reshape(29760*40*40,1)
Y_train1=Y_train1.reshape(29760*40*40,1)



In [1]:
for i in range(100):
    print(X_train[i],Y_train[i])

NameError: name 'X_train' is not defined

In [ ]:


from sklearn import svm

model = None
accuracy = None
############# Put your code here ################
model=svm.SVR()
model.fit(X_train1, Y_train1)
y_pred=model.predict(X_test1)
for i in range(100):
    print(y_pred[i])
    print(Y_train1[i])
accuracy=(Y_test1.shape[0]-np.count_nonzero(y_pred-Y_test1))/Y_test1.shape[0]      
################################################# 
print('Test data prediction accuracy (CART) : %f'%accuracy)

In [24]:
from sklearn.metrics import f1_score

def mae(y_true, y_pred) :
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    over_threshold = y_true >= 0.1
    
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    y_true = y_true.reshape(1, -1)[0]
    
    y_pred = y_pred.reshape(1, -1)[0]
    
    remove_NAs = y_true >= 0
    
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

In [25]:
model.compile(loss="mae", optimizer="adam",metrics=[maeOverFscore_keras,fscore_keras])

In [26]:
model_history = model.fit(train_dataset, epochs = 2, verbose=1)

Epoch 1/2
      9/Unknown - 276s 31s/step - loss: 0.5611 - maeOverFscore_keras: 9.2110 - fscore_keras: 0.2139

KeyboardInterrupt: 

In [1]:
from sklearn.tree import DecisionTreeClassifier


In [2]:
pred = model.predict(X_test)

NameError: name 'model' is not defined

In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission.iloc[:,1:] = pred.reshape(-1, 1600)

In [ ]:
submission.to_csv('Dacon_baseline.csv', index = False)